In [17]:
import pandas as pd
import numpy as np
import os
import math
import datetime
from urllib.request import urlopen as uReq, Request
from bs4 import BeautifulSoup as soup
from tqdm import tqdm
!pip install schedule  
import schedule
import time
from datetime import date
from datetime import datetime
from mimetypes import MimeTypes
from datetime import timedelta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Cleaning Code**

In [18]:
data = pd.read_excel("/content/Scraped_and_Cleaned_and_Items_Links.xlsx", sheet_name = "Fornite_Scraped_Data_FNBR_21_10")

In [19]:
data['Introduction_Season'] = data['Introduction_Season'].apply(lambda x: x.split(":")[1])
data['Release_Date'] = data['Release_Date'].apply(lambda x: x.split(":")[1])
data['Last_Seen'] = data['Last_Seen'].apply(lambda x: x.split(":")[1])
data['Occurences'] = data['Occurences'].apply(lambda x: int(x.split(":")[1]))
data['Item_Name'] = data['Item_Name'].apply(lambda x: x.replace("\n",''))
data['VC_Cost'] = data['VC_Cost'].apply(lambda x: x.replace("\n",''))
data['Rating'] = data['Rating'].apply(lambda x:  x if "No" in x  else float(x.split(":")[1].split("/")[0]) )
data['Total_Vote'] = data['Total_Vote'].apply(lambda x: float(x.split(" ")[1]) if "from" in x else x )

In [20]:
data

,Unnamed: 0,Scrape_Date,Introduction_Season,Release_Date,Last_Seen,Occurences,Item_Name,Item_Type,Rarity,VC_Cost,Rating,Total_Vote
0,0,2022-10-21,"Chapter 1, Season 3",Never,Never,0,Dark Matter,Back Bling,Legendary,Tier 71 (S3),4.24,38.0
1,1,2022-10-21,"Chapter 1, Season 3",Never,Never,0,Dark Void,Back Bling,Legendary,Dark Vanguard,4.06,33.0
2,2,2022-10-21,"Chapter 1, Season 3",Never,Never,0,Dark Wings,Back Bling,Legendary,Ravage,4.7,117.0
3,3,2022-10-21,"Chapter 1, Season 3",Never,Never,0,Datafiber Chargepack,Back Bling,Legendary,Page 8 (S18),4.0,2.0
4,4,2022-10-21,"Chapter 1, Season 3",Never,Never,0,Deflector,Back Bling,Legendary,Vertex,3.68,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6906,6906,2022-10-21,"Chapter 2, Season 4",Oct 24th 2020,Oct 11th 2022,7,Toy Soldier Wraps,Bundle,Uncommon,400,5.0,1.0
6907,6907,2022-10-21,"Chapter 2, Season 6",May 26th 2021,May 26th 2021,1,Trae Young's Locker,Bundle,Uncommon,"1,700",No Rating,Â
6908,6908,2022-10-21,"Chapter 3, Season 1",Dec 21st 2021,Oct 14th 2022,17,White Rabbit,Bundle,Uncommon,800,4.67,6.0
6909,6909,2022-10-21,"Chapter 3, Season 1",Mar 16th 2022,Mar 18th 2022,3,Wild Card Wrap,Bundle,Uncommon,600,4.0,1.0


In [4]:
# from google.colab import files
# data.to_csv('Fornite_Cleaned_Data_FNBR_21_OCT_2022.csv')
# files.download('Fornite_Cleaned_Data_FNBR_21_OCT_2022.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Scraping Code**

In [5]:
# All_Links = pd.read_excel("/content/Scraped_and_Cleaned_and_Items_Links.xlsx", sheet_name = "All_Items_Links")
# All_Links = All_Links.iloc[ : , :1]
# All_Links = All_Links.squeeze()
# All_Links = All_Links.values.tolist()

In [6]:
# Scrape_Date = date.today()

In [7]:
# data_df = pd.DataFrame(columns=["Scrape_Date","Introduction_Season","Release_Date","Last_Seen","Occurences","Item_Name","Item_Type","Rarity","VC_Cost","Rating", "Total_Vote"])

In [8]:
# for i in All_Links:
#   try:
#     print(All_Links.index(i))
#     my_url = i
#     req = Request(my_url,headers={'User-Agent':'Mozilla/5.0'})
#     page_html = uReq(req).read()
#     page_soup = soup(page_html,"html.parser")
#     div_tags = page_soup.find("div", {"class": "col-md-9 col-12 item-full"})
#     Item_Name = div_tags.find("h1",{'class':'item-title'}).getText()
#     Rarity = div_tags.find('h3',{'class':'type-rarity-string'}).find_all('span')[0].getText()
#     Item_Type = div_tags.find('h3',{'class':'type-rarity-string'}).find_all('span')[1].getText()
#     for i in div_tags.find('div',{'class':'shop-data-container'}).find_all('p'):
#       if "Introduced" in i.getText():
#         Introduction_Season = i.getText()
#       elif "Release Date" in i.getText():
#         Release_Date = i.getText()
#       elif "Last Seen" in i.getText():
#         Last_Seen = i.getText()
#       elif "Occurrences" in i.getText():
#         Occurences = i.getText()
#     VC_Cost = div_tags.find("p",{'class':'item-price'}).getText()
#     Rating = div_tags.find("div",{'class':'col-md-6 col-12 rating-text'}).find_all('p')[0].getText()
#     Total_Vote =  div_tags.find("div",{'class':'col-md-6 col-12 rating-text'}).find_all('p')[1].getText()
#     data_df= data_df.append(pd.DataFrame(columns=["Scrape_Date","Introduction_Season","Release_Date","Last_Seen","Occurences","Item_Name","Item_Type","Rarity","VC_Cost","Rating", "Total_Vote"],
#                                   data = [[Scrape_Date,Introduction_Season,Release_Date,Last_Seen,Occurences,Item_Name,Item_Type,Rarity,VC_Cost,Rating,Total_Vote
#                                           ]]),ignore_index=True)
#   except Exception as e:
#     print(e)

In [9]:
# data_df

In [10]:
# from google.colab import files
# data_df.to_csv('Fornite_Scraped_Data_FNBR_21_OCT_2022.csv')
# files.download('Fornite_Scraped_Data_FNBR_21_OCT_2022.csv')

# **For Individual Link**

In [11]:
# my_url = f'https://fnbr.co/bundle/black-ice-legends-pack'
# req = Request(my_url,headers={'User-Agent':'Mozilla/5.0'})
# page_html = uReq(req).read()
# page_soup = soup(page_html,"html.parser") 

In [12]:
# div_tags = page_soup.find("div", {"class": "col-md-9 col-12 item-full"})

In [13]:
# Item_Name = div_tags.find("h1",{'class':'item-title'}).getText()
# Rarity = div_tags.find('h3',{'class':'type-rarity-string'}).find_all('span')[0].getText()
# Item_Type = div_tags.find('h3',{'class':'type-rarity-string'}).find_all('span')[1].getText()
# Release_Date = div_tags.find('div',{'class':'shop-data-container'}).find_all('p')[0].getText()
# Last_Seen = div_tags.find('div',{'class':'shop-data-container'}).find_all('p')[1].getText()
# Occurences = div_tags.find('div',{'class':'shop-data-container'}).find_all('p')[2].getText()
# VC_Cost = div_tags.find("p",{'class':'item-price'}).getText()
# Rating = div_tags.find("div",{'class':'col-md-6 col-12 rating-text'}).find_all('p')[0].getText()
# Total_Vote =  div_tags.find("div",{'class':'col-md-6 col-12 rating-text'}).find_all('p')[1].getText()

In [14]:
# for i in div_tags.find('div',{'class':'shop-data-container'}).find_all('p'):
#   # print(i.getText())
#   if "Introduced" in i.getText():
#     print(i.getText())
#   elif "Release Date" in i.getText():
#     print(i.getText())
#   elif "Last Seen" in i.getText():
#     print(i.getText())
#   elif "Occurrences" in i.getText():
#     print(i.getText())

In [15]:
# data_df= data_df.append(pd.DataFrame(columns=["Scrape_Date","Release_Date","Last_Seen","Occurences","Item_Name","Item_Type","Rarity","VC_Cost","Rating", "Total_Vote"],
#                                 data = [[Scrape_Date,Release_Date,Last_Seen,Occurences,Item_Name,Item_Type,Rarity,VC_Cost,Rating,Total_Vote
#                                         ]]),ignore_index=True)

In [16]:
# print(f"Item_Name : {Item_Name}")
# print(f"Rarity : {Rarity}")
# print(f"Item_Type : {Item_Type}")
# print(f"Release_Date : {Release_Date}")
# print(f"Last_Seen : {Last_Seen}")
# print(f"Occurences : {Occurences}")
# print(f"VC_Cost : {VC_Cost}")
# print(f"Rating : {Rating}")
# print(f"Total_Vote : {Total_Vote}")